### AIM: To identify and classify the intent of a given text into the pre-defined 5 categories.

In [1]:
import os
import pickle
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
from sklearn.metrics import confusion_matrix, classification_report


RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")
     

In [3]:
train = train.append(valid).reset_index(drop=True)


In [4]:
train.shape


(13784, 2)

In [5]:
train.head()


,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist


In [6]:
#!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [41]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [42]:
os.makedirs("model", exist_ok=True)

In [43]:
!mv uncased_L-12_H-768_A-12/ model

In [44]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

# Data Preprocessing

In [15]:
class IntentDetectionData:
    DATA_COLUMN = "text"
    LABEL_COLUMN = "intent"

    def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes
    
        train, test = map(lambda df: df.reindex(df[IntentDetectionData.DATA_COLUMN].str.len().sort_values().index), [train, test])
    
        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
    
        for _, row in tqdm(df.iterrows()):
            text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))

        return np.array(x), np.array(y)

    def _pad(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)


In [45]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))


In [ ]:
#tokens = tokenizer.tokenize("I can't wait to meet my best friend!")


In [ ]:
#tokenizer.convert_tokens_to_ids(tokens)


# Model Creation

In [17]:
def create_model(max_seq_len, bert_ckpt_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
        
    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)
        
    return model

In [18]:
classes = train.intent.unique().tolist()
#print(classes)

In [19]:
data = IntentDetectionData(train, test, tokenizer, classes, max_seq_len=128)


13784it [00:01, 7284.27it/s]
/tmp/ipykernel_7932/3736353746.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(x), np.array(y)
700it [00:00, 7005.05it/s]


max seq_len 38


In [20]:
data.train_x.shape


(13784, 38)

In [21]:
data.train_x[0]


array([ 101, 2377, 3769,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [22]:
data.train_y[0]

0

In [23]:
data.max_seq_len


38

In [24]:
model = create_model(data.max_seq_len, bert_ckpt_file)


2023-04-04 18:02:13.680705: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-04 18:02:13.680779: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-04 18:02:13.680845: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hb-yogesh): /proc/driver/nvidia/version does not exist
2023-04-04 18:02:13.698702: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-04-04 18:02:13.911159: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3600000000 Hz
2023-04-04 18:02:13.912390: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fef20000b60 initialized for platform Host (this does not guarantee that 

bert shape (None, 38, 768)


2023-04-04 18:02:19.015976: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 93763584 exceeds 10% of free system memory.


Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7ff00dae52b0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


2023-04-04 18:02:30.757791: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 93763584 exceeds 10% of free system memory.


In [25]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 38)]              0         
_________________________________________________________________
bert (BertModelLayer)        (None, 38, 768)           108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 5383  

In [26]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

In [27]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(x=data.train_x, y=data.train_y,validation_split=0.1,batch_size=16,shuffle=True,epochs=2,
                    callbacks=[tensorboard_callback])
     

2023-04-04 18:02:31.313085: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.


Epoch 1/2
  1/776 [..............................] - ETA: 0s - loss: 1.9628 - acc: 0.1250

2023-04-04 18:03:00.188716: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2023-04-04 18:03:08.183816: I tensorflow/core/profiler/rpc/client/save_profile.cc:168] Creating directory: log/intent_detection/20230404-18021680611551/train/plugins/profile/2023_04_04_18_03_08
2023-04-04 18:03:08.349228: I tensorflow/core/profiler/rpc/client/save_profile.cc:174] Dumped gzipped tool data for trace.json.gz to log/intent_detection/20230404-18021680611551/train/plugins/profile/2023_04_04_18_03_08/hb-yogesh.trace.json.gz
2023-04-04 18:03:08.422256: I tensorflow/core/profiler/utils/event_span.cc:288] Generation of step-events took 1.12 ms

2023-04-04 18:03:09.657119: I tensorflow/python/profiler/internal/profiler_wrapper.cc:87] Creating directory: log/intent_detection/20230404-18021680611551/train/plugins/profile/2023_04_04_18_03_08Dumped tool data for overview_page.pb to log/intent_detection/20230404-18021680611551/train/plugins/profile/2023_04_04_18_03_08/hb-yoges

776/776 [==============================] - 2133s 3s/step - loss: 1.3115 - acc: 0.8676 - val_loss: 1.1724 - val_acc: 0.9920
Epoch 2/2
776/776 [==============================] - 2069s 3s/step - loss: 1.1852 - acc: 0.9817 - val_loss: 1.1701 - val_acc: 0.9949


In [28]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print("test acc", test_acc)

22/22 [==============================] - 24s 1s/step - loss: 1.1866 - acc: 0.9786
train acc 0.9904962182044983
test acc 0.9785714149475098


In [29]:
y_pred = model.predict(data.test_x).argmax(axis=-1)

In [30]:
print(classification_report(data.test_y, y_pred, target_names=classes))

                      precision    recall  f1-score   support

           PlayMusic       0.98      0.98      0.98        86
       AddToPlaylist       0.99      1.00      1.00       124
            RateBook       1.00      1.00      1.00        80
SearchScreeningEvent       0.98      0.93      0.96       107
      BookRestaurant       0.98      1.00      0.99        92
          GetWeather       1.00      0.98      0.99       104
  SearchCreativeWork       0.93      0.96      0.94       107

            accuracy                           0.98       700
           macro avg       0.98      0.98      0.98       700
        weighted avg       0.98      0.98      0.98       700



# Testing the trained Model with unseen data

In [31]:
sentences = ["play the song on high volume.","This book was awful"]

pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))

pred_token_ids = [tids + [0] * (data.max_seq_len - len(tids)) for tids in pred_token_ids]
pred_token_ids = np.array(list(pred_token_ids))

predictions = model.predict(pred_token_ids).argmax(axis=-1)

for text, label in zip(sentences, predictions):
    print("text:", text, "\nintent:", classes[label])
    print()
     

text: play the song on high volume. 
intent: PlayMusic

text: This book was awful 
intent: RateBook



In [46]:
# model.save("my_model.h5")
model.save('my_model.h5', save_format='h5')


In [36]:
print(classes)

['PlayMusic', 'AddToPlaylist', 'RateBook', 'SearchScreeningEvent', 'BookRestaurant', 'GetWeather', 'SearchCreativeWork']
